In [22]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [5]:
user_movie_likes = pd.read_csv('user_movie_likes.csv', header=None)

You are given a [CSV file](user_movie_likes.csv) with two columns: UserID and MovieID. Each row lists a user and a movie they liked

In [6]:
user_movie_likes.head()

,0,1
0,412,512
1,458,770
2,185,37
3,137,701
4,190,870


In [8]:
len(user_movie_likes[0].unique())

365

In [9]:
len(user_movie_likes[1].unique())

686

In [10]:
user_movie_likes.shape

(15000, 2)

1) Write a function that will take a MovieID and will return three movies that are similar to it (based on the similiarities in user likes).

2) Write a function that will take a UserID and recommend three movies based on what they have liked.

This an open-ended problem. Come up with a simple metric and just code it up. Come see me if you have any Qs!

In [17]:
user_movie_likes['like'] = 1
user_movie_likes.rename(columns={0:'user', 1:'movie'}, inplace=True)
user_movie_likes.head()

,user,movie,like
0,412,512,1
1,458,770,1
2,185,37,1
3,137,701,1
4,190,870,1


In [89]:
movie_matrix = user_movie_likes.pivot_table(aggfunc='count', index='movie', columns='user', values='like').fillna(0)
movie_matrix.head()

user,0,2,3,5,6,8,9,10,11,14,...,487,488,490,491,492,493,494,495,496,499
movie,,,,,,,,,,,,,,,,,,,,,
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [90]:
def movie_similarity(movie, movie_index):
    selected_movie = np.array(movie.loc[movie_index, :]).reshape(1,-1)
    all_other_movies = movie.drop(index=movie_index)
    
    print(selected_movie.shape, all_other_movies.shape)
    
    movie_results = cosine_similarity(all_other_movies, selected_movie) 
    
    return pd.DataFrame(movie_results).sort_values(0, ascending=False).head(3)
    

In [91]:
movie_similarity(movie_matrix, 236)

(1, 365) (685, 365)


,0
596,0.413167
90,0.382518
142,0.380609


In [85]:
# give me a user
# find all movies they like
# get top 3 movies for each movie
# return top 3 movies with an average

def user_suggestion(user_id):
    movies = user_movie_likes.loc[user_movie_likes.user==user_id, 'movie'] # series of movies
    suggested_movies = pd.DataFrame()
    for m in movies.values:
        suggested_movies = suggested_movies.append(movie_similarity(movie_matrix, m))
    
    suggested_movies.reset_index(inplace=True)
    suggested_movies.rename(columns={'index':'movie', 0:'similarity'}, inplace=True)
    
    suggested_movies = suggested_movies.groupby('movie')['similarity'].mean().sort_values(ascending=False)
    return suggested_movies.head(3)

In [88]:
user_suggestion(25)

movie
598    0.666667
597    0.629941
659    0.628935
Name: similarity, dtype: float64

In [62]:
user_movie_likes.groupby('user')['like'].sum().sort_values(ascending=False)

user
495    234
297    210
26     207
18     201
179    190
24     186
455    184
338    178
215    177
286    175
264    174
299    167
376    160
483    156
302    156
431    151
413    147
183    145
311    138
175    136
295    134
391    131
270    131
232    131
59     128
2      127
445    125
458    125
310    122
25     120
      ... 
168      2
158      1
287      1
149      1
247      1
163      1
228      1
477      1
227      1
166      1
273      1
128      1
292      1
23       1
43       1
63       1
425      1
409      1
396      1
74       1
437      1
383      1
125      1
358      1
332      1
307      1
303      1
421      1
290      1
188      1
Name: like, Length: 365, dtype: int64